## Reinforcement learning - TME 2 - Programmation Dynamique

L'objectif du TME est d'implémenter les algorithmes de programmation dynamique étudiés en cours (Policy Iteration et Value Iteration) et de les tester dans un framework classique (gym de open-ai, MDP GridWorld).

In [1]:
import matplotlib

matplotlib.use("TkAgg")
import gym
import gridworld
from gym import wrappers, logger
import numpy as np
import copy
from scipy.sparse import dok_matrix, lil_matrix

In [2]:
# For exemple 9, to construct recursively the MDP, we need to increase the recursion depth.
import sys
sys.setrecursionlimit(30000)

### Implémentation des algorithmes 

Trois agents sont implémentés :
+ RandomAgent : un agent aléatoire
+ PolicyIteration : un agent qui implémente l'algorithme Policy Iteration
+ ValueIteration : un agent qui implémente l'algorithme Value Iteration

Pour Policy Iteration et Value Iteration, on code une première implémentation naïve qui utilise la structure de données sous forme de dictionnaire. Cette implémentation naïve fonctionne en temps raisonnable pour tous les exemples donnés sauf l'exemple 9 (qui possède environ 200 000 états). Pour gérer cet exemple, on fait une implémentation plus sophistiquée utilisant des matrices sparse scipy.

In [3]:
env = gym.make("gridworld-v0")
env.setPlan("gridworldPlans/plan1.txt", {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1})
statedic, mdp = env.getMDP()

In [4]:
class RandomAgent(object):
    """The world's simplest agent!"""

    def __init__(self, env):
        self.action_space = env.action_space
        self.policy = 'Random policy'

    def act(self, observation, reward, done):
        return self.action_space.sample()
    
    def train(self, eps, gamma):
        pass

In [5]:
class PolicyIterationAgent(object):
    """Agent implementing Policy Iteration"""

    def __init__(self, env):
        self.env = env
        self.action_space = env.action_space
        self.statedic, self.mdp = env.getMDP()
        self.policy = {}
        for state, state_id in self.statedic.items():
            if state in self.mdp:
                list_actions = self.mdp[state].keys()
                self.policy[state_id] = self.action_space.sample()

    def act(self, observation, reward, done):
        return self.policy[self.statedic[self.env.state2str(observation)]]
    
    def value_evaluation(self, eps=5e-4, gamma=0.99):
        value = {}
        for state, state_id in self.statedic.items():
            value[state_id] = 0
            
        distance = np.inf
        while distance > eps:
            new_value = {}
            for state, state_id in self.statedic.items():
                if state in self.mdp:
                    new_value[state_id] = sum([proba*(reward + gamma*value[self.statedic[new_state]]) for proba, new_state, reward, done in self.mdp[state][self.policy[state_id]]])
                else:
                    new_value[state_id] = value[state_id]
            
            distance = np.linalg.norm(np.array(list(value.values()))-np.array(list(new_value.values())), ord=np.inf)
            value = new_value
        self.value = value
                
    def train(self, eps=5e-4, gamma=0.99):   # Policy Iteration algorithm
        policy_changed = True
        
        while policy_changed:
            
            policy_changed = False
            
            self.value_evaluation(eps, gamma)
            new_policy = {}
            
            for state, state_id in self.statedic.items():
                if state in self.mdp:
                    results = [sum([proba*(reward + gamma*self.value[self.statedic[new_state]]) for (proba, new_state, reward, done) in transitions]) for action, transitions in self.mdp[state].items()]
                    new_policy[state_id] = np.argmax(results)
                    if new_policy[state_id] != self.policy[state_id]:
                        policy_changed = True
            
            self.policy = new_policy

In [6]:
class ValueIterationAgent(object):
    """Agent implementing Value Iteration. Naive implementation with dictionary structure."""

    def __init__(self, env):
        self.env = env
        self.action_space = env.action_space
        self.statedic, self.mdp = env.getMDP()
        self.policy = {}
        for state, state_id in self.statedic.items():
            if state in self.mdp:
                list_actions = self.mdp[state].keys()
                self.policy[state_id] = self.action_space.sample()

    def act(self, observation, reward, done):
        return self.policy[self.statedic[self.env.state2str(observation)]]
                
    def train(self, eps=5e-4, gamma=0.99):  # Value Iteration algorithm
        value = {}
        for state, state_id in self.statedic.items():
            value[state_id] = 0
            
        distance = np.inf
        while distance > eps:
            new_value = {}
            
            for state, state_id in self.statedic.items():
                if state in self.mdp:
                    results = [sum([proba*(reward + gamma*value[self.statedic[new_state]]) for (proba, new_state, reward, done) in transitions]) for action, transitions in self.mdp[state].items()]
                    new_value[state_id] = np.max(results)
                else:
                    new_value[state_id] = value[state_id]
                    
            distance = np.linalg.norm(np.array(list(value.values()))-np.array(list(new_value.values())), ord=np.inf)
            value = new_value
                    
        for state, state_id in self.statedic.items():
                if state in self.mdp:
                    results = [sum([proba*(reward + gamma*value[self.statedic[new_state]]) for (proba, new_state, reward, done) in transitions]) for action, transitions in self.mdp[state].items()]
                    self.policy[state_id] = np.argmax(results)

In [7]:
class OptimizedValueIterationAgent(object):
    """Agent implementing Value Iteration with an efficient implementation (scipy sparse matrices)."""

    def __init__(self, env):
        self.env = env
        self.statedic, self.mdp = env.getMDP()
        self.policy = np.zeros(len(self.statedic.items()))
        
        # Translation of the MDP as CSC scipy matrices
        # See https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html#scipy.sparse.csc_matrix
        rewards = [lil_matrix((env.nS, env.nS)) for a in range(env.nA)]
        probas = [lil_matrix((env.nS, env.nS)) for a in range(env.nA)]
        for state in self.mdp.keys():
            for action, transitions in self.mdp[state].items():
                for (proba, new_state, reward, done) in transitions:
                    rewards[action][self.statedic[state], self.statedic[new_state]] = reward
                    probas[action][self.statedic[state], self.statedic[new_state]] += proba
                    
        self.rewards = [x.tocsc() for x in rewards]
        self.probas = [x.tocsc() for x in probas]

    def act(self, observation, reward, done):
        return self.policy[self.statedic[self.env.state2str(observation)]]
                
    def train(self, eps=5e-4, gamma=0.99):
        nS = self.env.nS
        nA = self.env.nA
        value = np.zeros(nS)
        distance = np.inf
        while distance > eps:
            new_value = np.zeros(nS)

            action_values = np.zeros((nS, nA))
            for a in range(nA):
                reward = self.rewards[a].copy()
                proba = self.probas[a]
                
                # create the array of gamma*value of the appropriate shape to be added to the rewards
                delta = np.repeat(gamma*value, np.diff(reward.indptr))
                reward.data += delta
                action_values[:, a] = (proba.multiply(reward)).sum(axis=1).flatten()

            new_value = np.max(action_values, axis=1)

            distance = np.linalg.norm(new_value-value, ord=np.inf)
            value = new_value

        action_values = np.zeros((nS, nA))
        for a in range(nA):
            reward = self.rewards[a].copy()
            proba = self.probas[a].copy()

            delta = np.repeat(gamma*value, np.diff(reward.indptr))
            reward.data += delta
            action_values[:, a] = (proba.multiply(reward)).sum(axis=1).flatten()


        self.policy = np.argmax(action_values, axis=1)

### Test sur la plateforme gym - problème GridWorld 

On commence par encapsuler le protocole de test dans des fonctions. Ces fonctions techniques ne présentent pas d'intérêt algorithmique.

In [8]:
def create_env(env_number, rewards):
    env = gym.make("gridworld-v0")
    env.setPlan("gridworldPlans/plan" + str(env_number) + ".txt", rewards)
    env.seed(0)  # Initialise le seed du pseudo-random
            
    return env

In [9]:
def run_experiment(env, agent, n_runs=10, verbose=1):
    outdir = 'gridworld-v0/agent-results'
    envm = wrappers.Monitor(env, directory=outdir, force=True, video_callable=False)
    reward = 0
    done = False
    rsum = 0
    rtot = 0
    for i in range(n_runs):
        obs = envm.reset()
        if verbose > 1:
            env.render(0.1)
        j = 0
        rsum = 0
        while True:
            action = agent.act(obs, reward, done)
            obs, reward, done, _ = envm.step(action)
            rsum += reward
            j += 1
            if verbose > 1:
                env.render()
            if done:
                if verbose > 1:
                    print("Episode : " + str(i) + " rsum=" + str(rsum) + ", " + str(j) + " actions")
                break
        rtot += rsum

    if verbose:
        print("Mean reward for agent %s over %d runs: %.2f" % (agent.__class__.__name__, n_runs, rtot/n_runs))
    env.close()
    return rtot

In [10]:
def test_agent(env_number, rewards, gamma=0.99, AgentClass=ValueIterationAgent, run_exp=True, n_runs=10, verbose=3, eps=5e-4):
    env = create_env(env_number, rewards)
    agent = AgentClass(env)
    if verbose > 1:
        print('MDP loaded')
    agent.train(eps=eps, gamma=gamma)
    if verbose:
        print('Test - Env %d - %s - %d runs' % (env_number, AgentClass.__name__, n_runs))
    if verbose > 2:
        print('Policy of ' + agent.__class__.__name__ +  ': ' +str(agent.policy))
    if run_exp:
        run_experiment(env, agent, n_runs, verbose)
    if verbose:
        print(' ')

On vérifie que Policy Iteration et Value Iteration ont la même policy. On affiche 10 runs pour vérifier que l'agent a bien le comportement attendu.

In [11]:
test_agent(0, {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1}, AgentClass=ValueIterationAgent, run_exp=False)
test_agent(0, {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1}, AgentClass=PolicyIterationAgent, run_exp=True, n_runs=5)

MDP loaded
Test - Env 0 - ValueIterationAgent - 10 runs
Policy of ValueIterationAgent: {0: 0, 2: 2, 3: 2, 4: 3, 6: 3, 7: 3, 8: 1, 9: 1, 10: 2}
 
MDP loaded
Test - Env 0 - PolicyIterationAgent - 5 runs
Policy of PolicyIterationAgent: {0: 0, 2: 2, 3: 2, 4: 3, 6: 3, 7: 3, 8: 1, 9: 1, 10: 2}
Episode : 0 rsum=0.974, 27 actions
Episode : 1 rsum=0.983, 18 actions
Episode : 2 rsum=0.982, 19 actions
Episode : 3 rsum=0.986, 15 actions
Episode : 4 rsum=0.989, 12 actions
Mean reward for agent PolicyIterationAgent over 5 runs: 0.98
 


En changeant les rewards (-0.4 pour un mouvement sur une case vide au lieu de -0.001), on observe que la policy optimale change.

In [17]:
test_agent(0, {0: -0.4, 3: 1, 4: 1, 5: -1, 6: -1}, run_exp=True, n_runs=5)

MDP loaded
Test - Env 0 - ValueIterationAgent - 5 runs
Policy of ValueIterationAgent: {0: 2, 2: 1, 3: 1, 4: 3, 6: 3, 7: 3, 8: 1, 9: 1, 10: 3}
Episode : 0 rsum=-0.20000000000000018, 4 actions
Episode : 1 rsum=-0.20000000000000018, 4 actions
Episode : 2 rsum=-0.20000000000000018, 4 actions
Episode : 3 rsum=-1.7999999999999998, 8 actions
Episode : 4 rsum=-1, 1 actions
Mean reward for agent ValueIterationAgent over 5 runs: -0.68
 


On peut comparer le reward moyen de l'agent optimal à celui de l'agent aléatoire. Comme prévu, l'agent optimal fait mieux que l'agent aléatoire.

In [13]:
test_agent(0, {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1}, run_exp=True, n_runs=1000, verbose=1)
test_agent(0, {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1}, AgentClass=RandomAgent, run_exp=True, n_runs=1000, verbose=1)

Test - Env 0 - ValueIterationAgent - 1000 runs
Mean reward for agent ValueIterationAgent over 1000 runs: 0.98
 
Test - Env 0 - RandomAgent - 1000 runs
Mean reward for agent RandomAgent over 1000 runs: -0.75
 


Pour les environnements 4 et 7, un phénomène intéressant se produit : il faut passer par un malus pour arriver à la sortie. Dans ces cas, il faut bien choisir la combinaison de gamma et du reward des cases vides (`reward[0]`) pour que l'agent apprenne. Par exemple, la combinaison `reward[0] = -0.001` et `gamma=0.99` ne fonctionne pas. Les combinaisons `reward[0] = -0.01` et `gamma=0.99` ou `reward[0] = -0.001` et `gamma=1` fonctionnent (mais donnent des comportements plus ou moins prudents dans le cas de l'environnement 7).

In [19]:
test_agent(4, {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1}, run_exp=True, n_runs=1, verbose=2, gamma=1)

MDP loaded
Test - Env 4 - ValueIterationAgent - 1 runs
Episode : 0 rsum=-0.02599999999999758, 28 actions
Mean reward for agent ValueIterationAgent over 1 runs: -0.03
 


In [20]:
test_agent(7, {0: -0.01, 3: 1, 4: 1, 5: -1, 6: -1}, run_exp=True, n_runs=5, verbose=2, gamma=0.99)

MDP loaded
Test - Env 7 - ValueIterationAgent - 5 runs
Episode : 0 rsum=2.4699999999999993, 58 actions
Episode : 1 rsum=2.4799999999999995, 57 actions
Episode : 2 rsum=-0.07000000000000006, 9 actions
Episode : 3 rsum=0.8999999999999999, 13 actions
Episode : 4 rsum=0.8799999999999999, 15 actions
Mean reward for agent ValueIterationAgent over 5 runs: 1.33
 


Enfin, pour l'environnement 9, on utilise l'agent OptimizedValueIterationAgent. L'exécution du test nécessite environ 2 minutes et 4 GB de mémoire vive. L'essentiel de ce budget est consacré à la construction du MDP et sa traduction sous la bonne forme de données (matrices sparse). La phase d'entraînement à proprement parler est très rapide (~ 10 secondes).

In [14]:
test_agent(9, {0: -0.01, 3: 1, 4: 1, 5: -1, 6: -1}, AgentClass=OptimizedValueIterationAgent, run_exp=True, n_runs=2, verbose=2, gamma=0.99)

MDP loaded
Test - Env 9 - OptimizedValueIterationAgent - 2 runs
Episode : 0 rsum=4.510000000000008, 155 actions
Episode : 1 rsum=4.730000000000006, 133 actions
Mean reward for agent OptimizedValueIterationAgent over 2 runs: 4.62
 
